## Service Setup

This notebooks sets up and verifies the services needed for the Document Intelligence app. The services include:

1. Storage Service (Volumes)
2. Database Service (Lakebase)
3. Document Service (DBSQL)
4. Agent Service (Model Serving)

These services are orchestrated by the application, but this notebook sets up and verifies each service individually.

We use the workspace client extensively throughout the app and repo, so first thing is to validate our connection.

In [1]:
from databricks.sdk import WorkspaceClient
from doc_intel.utils import get_workspace_client
client = get_workspace_client()
user_id = client.current_user.me().id

In [2]:
from doc_intel.config import DocConfig
config = DocConfig("./config.yaml")

## Database Service
The Database Service manages the Lakebase Postgres database. It handles automated schema creation, provides dynamic credential generation via Databricks SDK, and leverages the PGvector extension for semantic similarity search on small batches of documents. 

In [3]:
from doc_intel.database import DatabaseService
db_service = DatabaseService(client, config)

This code sets up the database instance. It takes about 5 minutes to spin up.

In [4]:
db_service.setup_database_instance()

True

Now that we have our database instance, we can get cooking with Postgres. Because it is an OLTP database, getting it setup is pretty fast.

In [5]:
db_service.connection_live

True

This method automates the creation of the database tables using Pydantic models, which can also be used in python and app code to validate the data before it is inserted into the database.

In [6]:
db_service.create_tables()
db_service.list_tables()

['chunks', 'conversations', 'documents', 'messages', 'users']

### User Table

In [7]:
db_service.create_user()

User(id='7873535765378608', username='scott.mckean@databricks.com', created_at=datetime.datetime(2025, 9, 4, 4, 26, 38, 803639, tzinfo=datetime.timezone.utc))

In [8]:
db_service.run_pg_query("SELECT * FROM users", return_dataframe=True)

,id,username,created_at
0,7873535765378608,scott.mckean@databricks.com,2025-09-04 04:26:38.803639+00:00


In [9]:
db_service.user_exists()

True

### Document Table

In [10]:
result = db_service.create_document(
    raw_path='/storage/documents/raw/financial_report_2022.pdf',
    processed_path='/storage/documents/processed/financial_report_2024.json',
    metadata={
        'filename': 'financial_report_2024.pdf',
        'file_size': 2048000,
        'pages': 25,
        'document_type': 'financial_statement',
        'company': 'Acme Corp',
        'year': 2024
    }
)
print(f'Document created: {result}')

ERROR:doc_intel.database.service:Failed to create document: duplicate key value violates unique constraint "documents_raw_path_key"
DETAIL:  Key (raw_path)=(/storage/documents/raw/financial_report_2022.pdf) already exists.



Document created: None


In [11]:
docs = db_service.run_pg_query("SELECT * FROM documents", return_dataframe=True)
docs

,id,user_id,raw_path,processed_path,metadata,created_at
0,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,7873535765378608,/storage/documents/raw/financial_report_2024.pdf,/storage/documents/processed/financial_report_...,"{'year': 2024, 'pages': 25, 'company': 'Acme C...",2025-09-04 04:26:49.699622+00:00
1,44649251-da36-4c36-bf71-bd078ba60bd3,7873535765378608,/storage/documents/raw/financial_report_2022.pdf,/storage/documents/processed/financial_report_...,"{'year': 2024, 'pages': 25, 'company': 'Acme C...",2025-09-04 04:37:24.118540+00:00
2,056f7e1f-cc88-43ac-b3b3-e8ae6592f8b5,7873535765378608,/example/document.pdf,/example/document_processed.json,"{'year': 2024, 'pages': 4, 'company': 'Example...",2025-09-04 04:46:45.218879+00:00
3,431796ab-5e75-4500-b757-9f3e719b0e2f,7873535765378608,/example/document_1756961232.pdf,/example/document_1756961232_processed.json,"{'year': 2024, 'pages': 4, 'company': 'Example...",2025-09-04 04:47:13.995466+00:00
4,0e48bf61-e2f9-43b7-95c3-b5219c74eb7a,7873535765378608,/example/simple_doc.pdf,/example/simple_doc_processed.json,"{'pages': 1, 'filename': 'simple_doc.pdf'}",2025-09-04 04:47:36.844494+00:00


In [12]:
db_service.get_document_by_id(docs.id.iloc[0], return_dataframe=True)

,id,user_id,raw_path,processed_path,metadata,created_at
0,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,7873535765378608,/storage/documents/raw/financial_report_2024.pdf,/storage/documents/processed/financial_report_...,"{'year': 2024, 'pages': 25, 'company': 'Acme C...",2025-09-04 04:26:49.699622+00:00


In [13]:
db_service.get_user_documents()

[Document(id='0e48bf61-e2f9-43b7-95c3-b5219c74eb7a', user_id='7873535765378608', raw_path='/example/simple_doc.pdf', processed_path='/example/simple_doc_processed.json', metadata={'pages': 1, 'filename': 'simple_doc.pdf'}, created_at=datetime.datetime(2025, 9, 4, 4, 47, 36, 844494, tzinfo=datetime.timezone.utc)),
 Document(id='431796ab-5e75-4500-b757-9f3e719b0e2f', user_id='7873535765378608', raw_path='/example/document_1756961232.pdf', processed_path='/example/document_1756961232_processed.json', metadata={'year': 2024, 'pages': 4, 'company': 'Example Corp', 'filename': 'sample_document.pdf', 'file_size': 1024000, 'document_type': 'financial_report'}, created_at=datetime.datetime(2025, 9, 4, 4, 47, 13, 995466, tzinfo=datetime.timezone.utc)),
 Document(id='056f7e1f-cc88-43ac-b3b3-e8ae6592f8b5', user_id='7873535765378608', raw_path='/example/document.pdf', processed_path='/example/document_processed.json', metadata={'year': 2024, 'pages': 4, 'company': 'Example Corp', 'filename': 'sampl

### Chunks

In [14]:
chunk_1 = {
    "content": "Your document content here...",
    "page_ids": ["1", "2"],  # Which pages this chunk came from
    "embedding": [0.1] * 768,  # 768-dimensional embedding vector
    "metadata": {
        "chunk_index": 0,
        "type": "paragraph",
        "word_count": 25,
        "section": "introduction"
    }
}

chunk_2 = {
    "content": "More document content here...",
    "page_ids": ["2"],  # Which pages this chunk came from
    "embedding": [0.2] * 768,  # 768-dimensional embedding vector
    "metadata": {
        "chunk_index": 0,
        "type": "paragraph",
        "word_count": 50,
        "section": "body"
    }
}

db_service.store_document_chunks(docs.id.iloc[0], [chunk_1, chunk_2])

True

In [15]:
db_service.get_document_chunks(docs.id.iloc[0], return_dataframe=True)

,id,doc_id,content,page_ids,embedding,metadata,created_at
0,457e7fbd-40fd-47dc-bba4-9790a527801f,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,Your document content here...,"[1, 2]","[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0...","{'type': 'paragraph', 'section': 'introduction...",2025-09-04 04:48:40.597569+00:00
1,556ecebe-91ae-4f28-aec0-c59524e6334e,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,Your document content here...,"[1, 2]","[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0...","{'type': 'paragraph', 'section': 'introduction...",2025-09-04 04:48:44.116711+00:00
2,984b978a-e9f4-4a75-a2e2-9cc7dc3fda2f,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,Your document content here...,"[1, 2]","[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0...","{'type': 'paragraph', 'section': 'introduction...",2025-09-04 04:49:33.435436+00:00
3,922cb9fa-cfaa-4934-8ce3-b5abccde4a90,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,More document content here...,[2],"[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0...","{'type': 'paragraph', 'section': 'body', 'word...",2025-09-04 04:49:33.671378+00:00
4,12068f24-e431-45c6-8058-eb9e0bf09542,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,Your document content here...,"[1, 2]","[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0...","{'type': 'paragraph', 'section': 'introduction...",2025-09-04 04:50:30.774321+00:00
5,0f5894df-71cd-494d-9c5e-8d514f9d637c,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,More document content here...,[2],"[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0...","{'type': 'paragraph', 'section': 'body', 'word...",2025-09-04 04:50:30.996838+00:00
6,f47f97f5-9fef-40e8-8a30-79bc829365cd,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,Your document content here...,"[1, 2]","[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0...","{'type': 'paragraph', 'section': 'introduction...",2025-09-04 04:55:51.165140+00:00
7,0d02e0d9-63ce-4002-a280-821285451505,fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913,More document content here...,[2],"[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0...","{'type': 'paragraph', 'section': 'body', 'word...",2025-09-04 04:55:51.371618+00:00


### Conversation


In [16]:
db_service.create_conversation(
    conversation_id="122",
    doc_ids=[docs.id.iloc[0]],
    metadata={
        "title": "Financial Report 2022"
    }
)

ERROR:doc_intel.database.service:Failed to create conversation: duplicate key value violates unique constraint "conversations_pkey"
DETAIL:  Key (id)=(122) already exists.



In [17]:
db_service.get_conversation_by_id("122")

{'id': '122',
 'user_id': '7873535765378608',
 'doc_ids': ['fdc37c40-b5f1-44d3-bd9a-ae9ae6e9c913'],
 'metadata': {'title': 'Financial Report 2022'},
 'created_at': datetime.datetime(2025, 9, 4, 4, 52, 10, 798466, tzinfo=datetime.timezone.utc),
 'updated_at': datetime.datetime(2025, 9, 4, 4, 52, 10, 798466, tzinfo=datetime.timezone.utc)}

In [19]:
# List the titles (and maybe summary) and the ids in the app service
[x['metadata']['title'] for x in db_service.get_user_conversations()]

['Financial Report 2022', 'Financial Report 2024', 'Financial Report 2024']

### Messages

In [23]:
db_service.add_message(
    conv_id="122",
    role="user", 
    content={"type": "text", "content": "Hello"},
    metadata={}
)


Message(id='a929d84e-812a-4b94-b546-b77b65ecee9b', conv_id='122', role='user', content={'type': 'text', 'content': 'Hello'}, metadata={}, created_at=datetime.datetime(2025, 9, 4, 4, 58, 24, 918362, tzinfo=datetime.timezone.utc))

In [24]:
db_service.get_conversation_messages("122")

[{'id': 'f168b86d-f179-489b-a815-9c1633f1d945',
  'conv_id': '122',
  'role': 'user',
  'content': {'type': 'text', 'content': 'Hello'},
  'metadata': {},
  'created_at': datetime.datetime(2025, 9, 4, 4, 58, 13, 516620, tzinfo=datetime.timezone.utc)},
 {'id': 'a929d84e-812a-4b94-b546-b77b65ecee9b',
  'conv_id': '122',
  'role': 'user',
  'content': {'type': 'text', 'content': 'Hello'},
  'metadata': {},
  'created_at': datetime.datetime(2025, 9, 4, 4, 58, 24, 918362, tzinfo=datetime.timezone.utc)}]